In [ ]:
## Run this pip installment to run programm
#pip install plotly

In [2]:
import numpy as np
from scipy import optimize
from scipy.optimize import Bounds, LinearConstraint
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import beta

In [3]:
# Define utility function given parameter; v = -2
def u(z, v=-2):
    return (z**(1+v)) / (1+v)

In [4]:
# Define a function that calculates optimal q for given parameters
def q_star(x=np.linspace(0.1,0.9), y=1, p=0.2):
    
    """
    
    This function returns the optimal coverage amount q
    given the values; x, y, p.
    
    ARGUMENTS:
    x (monetary loss): can be a 1d array
    y (value of assets in some numeraire): can be a float or int
    p (probability of observing the loss x): can be a float between 0 <= p <= 1
    
    """
    
    # Define expected utility function with free parameters q and x
    def V(q, x):
        return p * u(y - x + q - p*q) + (1 - p) * u(y - p*q)

    # initial value of q
    q_guess = 0
    
    # Optimize any q given constraint 0 <= q <= x for x in [0.1; 0.9] and q in [0;x]
    result = [] # Empty list to store results
    for i in x: # Loop through the different values of x
        
        # Define bounds for q
        bnds = ((0,i),)
        
        # Define objective function to be optimized, with free parameter q 
        obj_func = lambda q: -V(q, x=i)
        
        # Optimize obj func. 
        res = optimize.minimize(obj_func, q_guess, bounds = bnds)
        
        # Append optimal value q^star_i given 0.1 <= x_i <= 0.9
        result.append(round(res.x[0],7))
        
    
    # Return list containing different optimal values of q given value of x
    return result

In [6]:
## Plot q_star given x in range [0.1, 0.9]
y_axis = q_star()
x_axis = np.linspace(0.1,0.9)

fig = px.line(x = x_axis, y = y_axis, title='Optimal value of q given x',
             labels=dict(x ="$x$", y ="$q^\star$"))
fig.show()

In [8]:
# Define function that calculates acceptable contract from agents p.o.v given parameters
def pi_opt(x=0.6, y=1, p=0.2):
    
    """
    
    This function calculates the acceptable contracts, denoted \thilde{\pi}, given parameters.
    
    ARGUMENTS:
    x (monetary loss): can be a float
    y (value of assets in some numeraire): can be a float or int
    p (probability of observing the loss x): can be a float between 0 <= p <= 1
    
    """

    # Utility with insurance
    def V(pi, q):
        return p * u(y - x + q - pi) + (1 - p) * u(y - pi) - V_0

    # Set initial value of pi for maximization
    pi_guess = 0.2
    
    # Utility without insurance
    V_0 = p * u(y - x) + (1 - p) * u(y)
    
    # Optimize any pi_thilde given q in [0.1; 0.6]
    result = [] # Empty list to store results
    for i in np.linspace(0.1, 0.6): # Loop through the different values of q
        
        # Find pi thilde
        pi_thilde = optimize.root(V, pi_guess, args = (i))
        
        # Append results to empty list
        result.append(pi_thilde.x[0])
    
    # Return list containing different optimal values of pi given value of q
    return result  

In [9]:
## Plot pi_thilde given q in range [0.1, 0.6]
y_axis = np.array(pi_opt())
x_axis = np.linspace(0.1, 0.6)
fig2 = px.line(x = x_axis, y = y_axis, title = 'Optimal value of pi given q',
              labels=dict(x ="$q$", y ="$\pi$"))
fig2.show()

In [10]:
## Plot pi_thilde given q in range [0.1, 0.6] and pi[q,p]

# Define pi[q,p]
pi_pq = 0.2*np.linspace(0.1, 0.6)

# Create DF for plotting
df = pd.DataFrame(pi_pq, columns = ['$\pi[p;q]$'])
df['$\pi_{thilde}$'] = pi_opt()

# Plot DF
fig3 = px.line(df)
fig3.show()

In [11]:
# Define monte carlo simulation to find the integral
def MC_integral(gamma, pi, y=1):
    """
    
    This function finds the integral numerically by MC method.
    
    ARGUMENTS:
    gamma: can be a float
    pi: can be a float
    y= can be a float or int
    
    """
    
    # Set seed to get reproduceable results
    np.random.seed(2)
    
    # Empty list to store results
    result = []
    
    # Create loop with 10000 simulations of random draws from beta dist. with given parameters
    for i in range(10000):
        
        # Define x (random draw from beta dist.)
        x = np.random.beta(2, 7)
        
        # Define f(x) given boundaries
        f_x = beta.pdf(x, 2, 7)
        
        # Define value function
        value_func = u(y-(1-gamma)*x-pi)*f_x
        
        # Append each sim. to empty list
        result.append(value_func)
    
    # Returns mean of results of each sim.
    return np.array(result).mean()

In [12]:
# Print results given parameter values of gamma and pi
print(MC_integral(0.45,0.1))
print(MC_integral(0.9,0.2))

-2.887392732151206
-2.930543989299542
